# APS4 - Ciência dos Dados

Nome: Nevton de Castro

Nome: Eduardo Ghellere 

___
Carregando algumas bibliotecas:

In [7]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

In [8]:
with open('auth.pass') as fp:
    data = json.load(fp)
    
#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [10]:
#Produto escolhido:
produto = 'heineken'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [11]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [12]:
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:n])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[n:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()


## twetts classificados no excel.
    Separando-os entre irrelevantes e relevantes

In [27]:
dados = pd.read_excel('heineken.xlsx')
irrelevantes = dados[dados.valor==0] #tweets irrelevantes
relevantes = dados[dados.valor==1] #tweets relevantes
irrelevantes.valor.value_counts(), relevantes.valor.value_counts()
rel = relevantes['Treinamento'].tolist()
irr = irrelevantes['Treinamento'].tolist()
